In [1]:
%matplotlib inline
import pandas as pd
import numpy as np


df_train=pd.read_csv('train.csv',delimiter=';')
df_test=pd.read_csv('test.csv',delimiter=';')

In [2]:
#собираем вместе
df_train['train']=1
df_test ['train']=0
df_test['item_views']=None
df_all=pd.concat([df_train,df_test])


print (len(df_train), len(df_test), len(df_all))
df_all = df_all.sample(n = 100)

423772 418991 842763


In [3]:
del df_test, df_train

In [4]:
#час и день начала
df_all['startdate']=df_all['start_time'].apply(lambda x:int(x[8:10]))
df_all['starthourt']=df_all['start_time'].apply(lambda x:int(x[11:13]))

In [5]:
#вытаскиваем фичи из текста
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer(max_features=10000, ngram_range=(1,2))

In [6]:
#нормализация
def nomalize2(x):
    from pymorphy2 import MorphAnalyzer
    import re
    p=MorphAnalyzer()

    def nomalize1(x):
        t=p.parse(x)
        if len(x)>0:
            return t[0].normal_form
        return x
    
    x2=re.sub('\s+',' ',re.sub(u'[^а-яёa-z0-9\.]',' ',str(x).lower())).strip()
    
    return ' '.join([nomalize1(a) for a in x2.split(' ')])

In [7]:
#длина текста и количество слов
df_all['title_length']=df_all['title'].apply(lambda x:len(str(x)))
df_all['title_word_count']=df_all['title'].apply(lambda x:len(str(x).split(' ')))

In [8]:
# нормализую тайтлы
df_all['title_norm']=df_all.title.apply(lambda x: nomalize2(x))

In [9]:
#tf-idf
import datetime

dstart=datetime.datetime.now()
text_features=tfidf.fit_transform(df_all['title_norm'].values)
dend=datetime.datetime.now()

In [10]:
#доьбавляем часовой пояс
region=pd.read_csv('region.csv')

In [11]:
df_all=df_all.set_index('region').join(region.set_index('region')).reset_index()

In [12]:
#время по часовому поясу
df_all['real_time']=df_all['starthourt']+df_all['timezone']-3

In [13]:
#категориальные переменные
import datetime

tmp2=[]

#'param1','param2','param3'

for a in ['category','owner_type','region','subcategory','param1','param2','param3']:
    tmp=pd.get_dummies(df_all[a])
    tmp.columns=map(lambda x:a+'_'+x, tmp.columns)
    tmp2.append(tmp)
    print (a, datetime.datetime.now())

category 2017-02-19 21:23:15.031945
owner_type 2017-02-19 21:23:15.033239
region 2017-02-19 21:23:15.034572
subcategory 2017-02-19 21:23:15.036475
param1 2017-02-19 21:23:15.038979
param2 2017-02-19 21:23:15.042115
param3 2017-02-19 21:23:15.044157


In [14]:
#все вместе
df_all_2=pd.concat(tmp2+[df_all[['price','train','item_id','item_views',\
                                          'startdate','starthourt',\
                                         'real_time','title_length','title_word_count']]],axis=1)

In [15]:
del tmp2

In [16]:
#собираем это все в разряженную матрицу
import numpy as np
from scipy.sparse import hstack, csr_matrix
X=hstack([csr_matrix(df_all_2.drop(['train','item_id','item_views'],axis=1).values),\
          text_features])
is_train=df_all_2['train'].values
y=df_all_2[df_all_2['train']==1]['item_views'].values.astype(np.float)

In [17]:
print (df_all.columns)
id_name= df_all.columns[2]
ids=df_all[id_name]

Index(['region', 'category', 'id', 'item_id', 'item_views', 'owner_type',
       'param1', 'param2', 'param3', 'price', 'start_time', 'subcategory',
       'title', 'train', 'startdate', 'starthourt', 'title_length',
       'title_word_count', 'title_norm', 'timezone', 'real_time'],
      dtype='object')


import cPickle as pickle
f=open('/mnt/data/a_pickle.pk1','rb')
X,is_train,y,ids = pickle.load(f)
f.close()

In [18]:
# в разряженную матрицу
X=csr_matrix(X)
X_train=X[np.where(is_train==1)]
X_test=X[np.where(is_train==0)]

In [20]:
# обучаю бустинг для предсказания цены
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label = np.log1p(y))
dtest = xgb.DMatrix(X_test)
num_round = 6422

param={'colsample_bytree': 0.5,
 'eta': 0.01,
 'max_depth': 18,
 'objective': 'reg:linear',
 'silent': 0,
 'subsample': 1}

bst_final = xgb.train(param, dtrain, num_round)

In [21]:
# предсказываю цены для теста
pred=bst_final.predict(dtest)

In [22]:
res1=np.expm1(pred)

In [23]:
tmp2=df_all[df_all['train']==0][['id']]

In [24]:
tmp2['item_views']=res1
tmp2.to_csv('second_subm_3_fixed.csv',index=False)